In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## PLAYBOOK

THIS NOTEBOOK IS NOT ANY THEORY BUT I AM USING IT TO PLAY WITH DIFFERENT ALGORITHM


In [ ]:
import pandas as pd
import numpy as np

import sklearn

In [ ]:
data = pd.read_csv('/kaggle/input/insurance/insurance.csv')
data.head()

In [ ]:
# importing visualization libraries 

import seaborn as sns
import matplotlib.pyplot as plt 

In [ ]:
plt.subplots(1, 1, figsize=(18, 8))
sns.distplot(data['charges'],  color ='r', kde=True)
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(18, 12))

sns.countplot(data['children'], palette='rocket', ax=ax[0][0])
sns.countplot(data['smoker'], palette='rocket', ax=ax[0][1])
sns.countplot(data['sex'], palette='rocket', ax=ax[1][0])
sns.countplot(data['region'], palette='rocket', ax=ax[1][1])
plt.show()
fig, ax = plt.subplots(1, 1, figsize=(18, 6))
sns.distplot(data['bmi'],  color ='r', kde=True)
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(18, 12))

sns.violinplot(data['children'],data['charges'], palette='rocket', ax=ax[0][0])
sns.violinplot(data['smoker'],data['charges'], palette='rocket', ax=ax[0][1])
sns.violinplot(data['sex'], data['charges'] ,palette='rocket', ax=ax[1][0])
sns.violinplot(data['region'], data['charges'],palette='rocket', ax=ax[1][1])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 8))

sns.scatterplot(data['bmi'], data['charges'], color='r')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 8))

sns.scatterplot(data['age'], data['charges'], color='r')

In [ ]:
data.describe(include='all').T

In [ ]:
data.info()

In [ ]:
from sklearn import preprocessing 
for col in ['sex', 'smoker', 'region']:
    le = preprocessing.LabelEncoder()
    
    data[col] = le.fit_transform(data[col])
    
    

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(['charges'], axis=1), data['charges'], test_size=0.2, random_state=1)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
reg = make_pipeline(StandardScaler(),
                    LinearRegression())
reg.fit(X_train, y_train)
reg.score(X_test, y_test)

In [ ]:
reg = make_pipeline(StandardScaler(),
                    SGDRegressor(max_iter=10000, tol=1e-3, alpha=0.001, early_stopping=True))
reg.fit(X_train, y_train)

In [ ]:
reg.score(X_test, y_test)

In [ ]:
# corr matrix 



In [ ]:
corr = data.corr()

## Feature Engineering 

In [ ]:
import seaborn as sns 
fig, ax = plt.subplots(1, 1, figsize=(11, 9))

sns.heatmap(corr)

In [ ]:
reg = LinearRegression()
reg.fit(X_train[['smoker', 'bmi', 'age']], y_train)
reg.score(X_test[['smoker', 'bmi', 'age']], y_test)

In [ ]:
reg = make_pipeline(StandardScaler(),
                    SGDRegressor(max_iter=10000, alpha=0.0009, early_stopping=True))
reg.fit(X_train[['smoker', 'bmi', 'age']], y_train)
reg.score(X_test[['smoker', 'bmi', 'age']], y_test)

In [ ]:
import sklearn.metrics as metrics
print('MAE: {}'.format(metrics.mean_absolute_error(y_test, reg.predict(X_test[['smoker', 'bmi', 'age']]))))
print('MSE: {}'.format(metrics.mean_squared_error(y_test, reg.predict(X_test[['smoker', 'bmi', 'age']]))))
print('RMSE: {}'.format(np.sqrt(metrics.mean_squared_error(y_test, reg.predict(X_test[['smoker', 'bmi', 'age']])))))

In [ ]:
from itertools import combinations
columns_list = X_train.columns
interactions = list(combinations(columns_list, 2))

In [ ]:
interaction_dict = {}
for interaction in interactions:
    X_train_int = X_train
    X_train_int['int'] = X_train_int[interaction[0]] * X_train_int[interaction[1]]
    lr3 = LinearRegression()
    lr3.fit(X_train_int, y_train)
    interaction_dict[lr3.score(X_train_int, y_train)] = interaction

In [ ]:
top_5 = sorted(interaction_dict.keys(), reverse = True)[:5]
for interaction in top_5:
   print(interaction_dict[interaction])

In [ ]:
X_train_int = X_train
X_train_int['ad_ed'] = X_train_int['bmi'] * X_train_int['children']
X_train_int['bmi_smoker'] = X_train_int['bmi'] * X_train_int['smoker']
X_train_int['smoker_region'] = X_train_int['smoker'] * X_train_int['region']
X_train_int['sex_smoker'] = X_train_int['sex'] * X_train_int['smoker']
X_train_int['child_smoker'] = X_train_int['children'] * X_train_int['smoker']
X_train_int['child_region'] = X_train_int['children'] * X_train_int['region']





In [ ]:
lr4 = LinearRegression()
lr4.fit(X_train_int, y_train)
lr4.score(X_train_int, y_train)

In [ ]:
import sklearn.metrics as metrics
print('MAE: {}'.format(metrics.mean_absolute_error(y_train, lr4.predict(X_train_int))))
print('MSE: {}'.format(metrics.mean_squared_error(y_train, lr4.predict(X_train_int))))
print('RMSE: {}'.format(np.sqrt(metrics.mean_squared_error(y_train, lr4.predict(X_train_int)))))

In [ ]:
X_train_int['ad_+ed'] = X_train_int['bmi'] + X_train_int['children']
X_train_int['bmi_+smoker'] = X_train_int['bmi'] + X_train_int['smoker']
X_train_int['smoker_+region'] = X_train_int['smoker'] + X_train_int['region']
X_train_int['sex_+smoker'] = X_train_int['sex'] + X_train_int['smoker']
X_train_int['child_+smoker'] = X_train_int['children'] + X_train_int['smoker']
X_train_int['child_+region'] = X_train_int['children'] + X_train_int['region']

In [ ]:
X_train_int.head()

In [ ]:
lr4 = LinearRegression()
lr4.fit(X_train_int, y_train)
lr4.score(X_train_int, y_train)

In [ ]:
import sklearn.metrics as metrics
print('MAE: {}'.format(metrics.mean_absolute_error(y_train, lr4.predict(X_train_int))))
print('MSE: {}'.format(metrics.mean_squared_error(y_train, lr4.predict(X_train_int))))
print('RMSE: {}'.format(np.sqrt(metrics.mean_squared_error(y_train, lr4.predict(X_train_int)))))

In [ ]:
data = pd.read_csv('/kaggle/input/insurance/insurance.csv')
data.head()

In [ ]:
X = data.drop(['charges'], axis=1)
y = data['charges']

In [ ]:
X['sex+child'] = X['sex']+' '+str(X['children'])
X['sex+smoker'] = X['sex']+' '+X['smoker']
X['sex+region'] = X['sex']+' '+X['region']
X['child+smoker'] =str(X['children'])+' '+X['smoker']
X['child+region'] = str(X['children'])+' '+X['region']
X['smoker+region'] = X['smoker']+' '+X['region']





In [ ]:
from sklearn import preprocessing

for col in ['sex', 'smoker', 'region', 'sex+child', 'sex+smoker', 'sex+region', 'child+smoker', 'child+region','smoker+region' ]:
    le = preprocessing.LabelEncoder()
    
    X[col] = le.fit_transform(X[col])

In [ ]:
lr4 = LinearRegression()
lr4.fit(X, y)
lr4.score(X, y)

In [ ]:
X_train_int = X
X_train_int['ad_ed'] = X_train_int['bmi'] * X_train_int['children']
X_train_int['bmi_smoker'] = X_train_int['bmi'] * X_train_int['smoker']
X_train_int['smoker_region'] = X_train_int['smoker'] * X_train_int['region']
X_train_int['sex_smoker'] = X_train_int['sex'] * X_train_int['smoker']
X_train_int['child_smoker'] = X_train_int['children'] * X_train_int['smoker']
X_train_int['child_region'] = X_train_int['children'] * X_train_int['region']





In [ ]:
lr4 = LinearRegression()
lr4.fit(X_train_int, y)
lr4.score(X_train_int, y)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(11, 9))

sns.scatterplot(lr4.predict(X_train_int), y, color='r', ax=ax)
ax.set_xlabel('Predicted')
#sns.lineplot(lr4.predict(X_train_int), ax=ax)


In [ ]:
print('MAE: {}'.format(metrics.mean_absolute_error(y, lr4.predict(X_train_int))))
print('MSE: {}'.format(metrics.mean_squared_error(y, lr4.predict(X_train_int))))
print('RMSE: {}'.format(np.sqrt(metrics.mean_squared_error(y, lr4.predict(X_train_int)))))